In [1]:
from pathlib import Path
from kedro.framework.session import KedroSession

# If pyproject.toml exists in the current directory, use it;
# otherwise, assume the project root is the parent directory.
project_path = Path.cwd() if (Path.cwd() / "pyproject.toml").exists() else Path.cwd().parent

session = KedroSession.create(project_path=project_path, env="base")
context = session.load_context()
print("Project loaded from:", project_path)


[02/10/25 18:03:26] INFO     Using                                                                  ]8;id=654531;file:///usr/local/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=778114;file:///usr/local/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/usr/local/lib/python3.11/site-packages/kedro/framework/project/rich_                
                             logging.yml' as logging configuration.                                                

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 # otherwise, assume the project root is the parent directory.                               │
│    6 project_path = Path.cwd() if (Path.cwd() / "pyproject.toml").exists() else Path.cwd().pa    │
│    7                                                                                             │
│ ❱  8 session = KedroSession.create(project_path=project_path, env="base")                        │
│    9 context = session.load_context()                                                            │
│   10 print("Project loaded from:", project_path)                                                 │
│   11                                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.11/site-packages/kedro/framework/session/session.py:156 in create         │
│                                                                                                  │
│   153 │   │   Returns:                                                                           │
│   154 │   │   │   A new ``KedroSession`` instance.                                               │
│   155 │   │   """                                                                                │
│ ❱ 156 │   │   validate_settings()                                                                │
│   157 │   │                                                                                      │
│   158 │   │   session = cls(                                                                     │
│   159 │   │   │   project_path=project_path,                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.11/site-packages/kedro/framework/project/__init__.py:332 in               │
│ validate_settings                                                                                │
│                                                                                                  │
│   329 │   More info on the dynaconf issue: https://github.com/dynaconf/dynaconf/issues/460       │
│   330 │   """                                                                                    │
│   331 │   if PACKAGE_NAME is None:                                                               │
│ ❱ 332 │   │   raise ValueError(                                                                  │
│   333 │   │   │   "Package name not found. Make sure you have configured the project using "     │
│   334 │   │   │   "'bootstrap_project'. This should happen automatically if you are using "      │
│   335 │   │   │   "Kedro command line interface."                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Package name not found. Make sure you have configured the project using 'bootstrap_project'. This 
should happen automatically if you are using Kedro command line interface.

In [2]:
import click
from pathlib import Path
from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession
from kedro.framework.project import pipelines

from prefect import flow, get_run_logger




def create_kedro_flow(pipeline_name: str, env: str):
    """
    Returns a Prefect flow that bootstraps your Kedro project and executes the
    specified pipeline.
    """
    return run_kedro_pipeline

@flow(name="kedro_deployment_flow")
def run_kedro_pipeline():
    logger = get_run_logger()
    project_path = Path.cwd() if (Path.cwd() / "pyproject.toml").exists() else Path.cwd().parent
    # project_path = Path.cwd()
    
    # Bootstrap the Kedro project (loads metadata, etc.)
    metadata = bootstrap_project(project_path)
    logger.info("Project name: %s", metadata.project_name)
    
    # Create a Kedro session and load the project context.
    session = KedroSession.create(project_path=project_path, env=env)
    context = session.load_context()
    
    # Retrieve the pipeline from Kedro's pipeline registry.
    pipeline = pipelines.get(pipeline_name)
    if pipeline is None:
        logger.error("Pipeline '%s' not found.", pipeline_name)
        raise ValueError(f"Pipeline '{pipeline_name}' not found in the registry.")
    
    # Execute the pipeline.
    context.run(pipeline_name=pipeline_name)
    logger.info("Kedro pipeline '%s' executed successfully.", pipeline_name)




In [1]:
# Define a Click command to accept parameters from the command line.
@click.command()
@click.option(
    "-p", "--pipeline", "pipeline_name", default="simple_pipeline",
    help="The Kedro pipeline name to run."
)
@click.option(
    "--env", "-e", type=str, default="base",
    help="The Kedro environment to load the project with."
)
@click.option(
    "--deployment_name", default="simple_deployment",
    help="The name to give your Prefect deployment."
)
@click.option(
    "--work_pool_name", default="default-agent-pool",
    help="The work pool name that the deployment will use."
)
@click.option(
    "--version", default="1.0",
    help="The version string for this deployment."
)
@click.option(
    "--cron", default=None,
    help="(Optional) A cron schedule string (e.g., '0 * * * *') to schedule the flow."
)
def prefect_deploy(pipeline_name, env, deployment_name, work_pool_name, version, cron):
    """
    Registers a Kedro pipeline as a Prefect 3 flow deployment.
    
    This function creates a Prefect flow that bootstraps your Kedro project,
    loads the specified pipeline, and then executes it. It then creates a deployment
    using the from_source method (to specify the source directory and the entrypoint)
    and finally registers the deployment with the Prefect server.
    """
    # Create the Kedro flow (a Prefect flow wrapping your Kedro logic)
    kedro_flow = create_kedro_flow(pipeline_name, env)
    
    # Define the entrypoint. Here we assume this file (register_prefect_flow.py)
    # contains the flow function "run_kedro_pipeline".
    entrypoint = f"{Path(__file__).name}:run_kedro_pipeline"
    
    # Create a deployment from source.
    # The "source" tells Prefect where to find your flow code (here, the current directory),
    # and "entrypoint" specifies the file and function to load.
    deployment = kedro_flow.from_source(
        source=str(Path.cwd()),
        entrypoint=entrypoint
    )
    
    # Deploy the flow. Note that we no longer pass a "storage" parameter here.
    deployment.deploy(
        name=deployment_name,
        work_pool_name=work_pool_name,
        version=version,
        cron=cron  # Pass None if no schedule is needed.
    )
    click.echo(f"Deployment '{deployment_name}' applied successfully.")

NameError: name 'click' is not defined

In [ ]:
if __name__ == "__main__":
    prefect_deploy()

# python register_prefect_flow.py --pipeline simple_pipeline --env base --deployment_name simple_deployment --work_pool_name default-agent-pool --version 1.0 --cron "* * * * *"
